In [ ]:
from IPython.core.display import display, HTML,clear_output, Image
import ipywidgets as widgets
import rasterio
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def plot_location(df_c,df_o,lat,lon):
    c_df = df_c[df_c.intersects(Point(lon,lat))].copy()
    if len(c_df)!=0:
        c = c_df['CNTRY_NAME'].tolist()[0]
        display(HTML(f'<h3><left>The corrdinates are in {c}</left></h3>'))
        c_df['geometry'].plot(figsize=(5,5))
        plt.show(plt.scatter(lon,lat,color='r',s=200))
    else:
        o_df = df_o[df_o.intersects(Point(lon,lat))].copy()
        if len(o_df)!=0:
            o = o_df['name'].tolist()[0]
            display(HTML(f'<h2 style="background-color:powderblue;"><left>oops.. your coordinates are in the {o}</left></h2>'))
            display(Image(filename="fish.png"))
            return 0
        else:
            display(HTML('<h2 style="background-color:red;"><left>unknown country, your coordinates do not make any sense :)  please check again the coordinates</left></h2>'))
            display(Image(filename="stop_sign.png"))
            return 0
        

        
# def get_cc_per_band(r,lat,lon,i):
#     # https://gis.stackexchange.com/questions/221292/retrieve-pixel-value-with-geographic-coordinate-as-input-with-gdal

#     band = r.GetRasterBand(i)

#     cols = r.RasterXSize
#     rows = r.RasterYSize

#     transform = r.GetGeoTransform()

#     xOrigin = transform[0]
#     yOrigin = transform[3]
#     pixelWidth = transform[1]
#     pixelHeight = -transform[5]

#     data = band.ReadAsArray(0, 0, cols, rows)

#     point = (lon, lat)

#     col = int((point[0] - xOrigin) / pixelWidth)
#     row = int((yOrigin - point[1] ) / pixelHeight)

#     return data[row][col]



# def get_cloud_cover(lat,lon,raster_path,status):
#     if status!=0:
# #         r = rasterio.open(raster_path)
        
#         driver = gdal.GetDriverByName('GTiff')
#         r = gdal.Open(raster_path)
    
#         cc = [get_cc_per_band(r,lat,lon,i+1) for i in range(r.RasterCount)]
#         dates = pd.date_range(start='2019-06-01', end='2022-04-01', freq='MS')   
#         df = pd.DataFrame([dates,cc],index=['Date','Cloud Cover']).T
#         df['Cloud Cover'] = df['Cloud Cover'].astype('int')
#         return df
#     else:
#         return 0



def get_cloud_cover(lat,lon,i,status):
    if status!=0:
        r = rasterio.open(i)
        cc = r.read(1)[r.index(lon,lat)]
        return {'Date':pd.to_datetime(i.split('_')[5]),
                'Cloud Cover': round(100*cc)}
    else:
        return 0






def plot_results(df,status):
    if status!=0:
        plt.figure(figsize=(15,5))
        plt.plot(df['Date'],df['Cloud Cover'],'o-')
        plt.ylim(0,100)
        plt.xticks(rotation=90, size=17) 
        plt.yticks(size=17)
        plt.ylabel('Cloud Cover %',size=20)
        plt.axhline(y=60, color='r', linestyle='-')
        plt.title('how much (at 10:30 AM local time on average) of that pixel (10X10km) was covered by clouds per month', size=15)
        plt.grid();
    else:
        print('no data')
        
        
raster_path = r"C:\Manna\general\cloud_tool\global_cloud_cover_2020_2022_int.tiff"
df_countries = gpd.read_file("countries.geojson")
df_oceans = gpd.read_file("Oceans_bounderies.geojson")

In [ ]:
display(HTML('<h1><left>Global Cloud Tool</left></h1>'))
display(HTML('<h4><left>Check cloud cover per coordinates</left></h4>'))
Image(filename="clouds.png")#,width=200
rasters_list = glob.glob('*TIFF*')

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [ ]:
# !jupyter serverextension enable voila --sys-prefix

In [ ]:
name = widgets.Text(placeholder='Please enter coordinates (lat,lon)')

In [ ]:
button_send = widgets.Button(
                description='Submit',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        lat,lon = float(name.value.split(',')[0]),float(name.value.split(',')[1])
        status = plot_location(df_countries,df_oceans,lat,lon)
#         plt.show(plt.scatter(lon,lat,color='r',s=200))
        plt.show(plot_results(pd.DataFrame([get_cloud_cover(lat,lon,i,status) for i in rasters_list]),status))

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [ ]:
text_1 = widgets.HTML(value="<h4>Please enter coordinates (lat,lon)</h4>")
vbox_text = widgets.VBox([text_1,name,vbox_result])

In [ ]:
page = widgets.HBox([vbox_text])
display(page)